<a href="https://colab.research.google.com/github/Frexile/Machine_Learning_ITMO/blob/main/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1-nskFrjNS5ScLBtpgjk8oURgu14ZNHTL"})   
downloaded.GetContentFile('copy_transactions.csv');
transactions = pd.read_csv(filepath_or_buffer = "copy_transactions.csv", sep = ',', nrows=1000000)

downloaded_tr_types = drive.CreateFile({"id":"12c2clOzcEgBLiykuWKb3jlqg8Ge1FpOW"})
downloaded_tr_types.GetContentFile('tr_types.csv');
tr_types = pd.read_csv(filepath_or_buffer="tr_types.csv", sep=';' )

downloaded_gender_train = drive.CreateFile({"id":"1EnzPjklkXe2iQdHCln8-fLb0Vobu7ETE"})
downloaded_gender_train.GetContentFile("gender_train.csv");
gender_train = pd.read_csv(filepath_or_buffer = "gender_train.csv", sep = ',')

downloaded_tr_mcc_codes = drive.CreateFile({"id":"1O6hBt9PrBV8l9VE4xL2PcannAjzSCJPl"})
downloaded_tr_mcc_codes.GetContentFile("tr_mcc_codes.csv");
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep = ';')

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [ ]:
random_sample = transactions.sample(n=1000)
random_sample

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
644673,86344405,379 14:24:18,6011,2010,-112295.79,329951,0.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
552693,67984042,84 09:24:02,6011,2010,-22459.16,NaN,NaN,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
530373,21276897,111 15:44:53,6011,7010,2156079.14,NaN,1.0,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в своем тер.банке)
301494,49148603,286 08:39:56,5499,1010,-4536.75,194679,0.0,"Различные продовольственные магазины — рынки, ...",Покупка. POS ТУ СБ РФ
602623,36483672,12 12:44:18,6011,2010,-80852.97,NaN,NaN,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
...,...,...,...,...,...,...,...,...,...
246087,85036381,340 00:00:00,5533,1110,-5839.38,77074034,1.0,Автозапчасти и аксессуары,Покупка. POS ТУ Россия
395388,17096973,348 13:35:35,5411,1010,-10266.75,852774,NaN,"Бакалейные магазины, супермаркеты",Покупка. POS ТУ СБ РФ
509613,82581973,405 00:00:00,5735,1200,-3795.60,NaN,NaN,Магазины звукозаписи,Покупка. Зарубеж. ТУ
699580,64294802,44 23:09:00,6011,2011,-44918.32,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанка (в других ТБ)


In [ ]:
tr_types

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...
...,...,...
150,2990,Списание по требованию
151,2992,Списание средств для погашения задолженности п...
152,3001,Комиссия за обслуживание ссудного счета
153,3030,Плата за обслуживание банковской карты (за пос...


In [ ]:
tr_types_sample=tr_types[tr_types.tr_description.str.contains("POS") | tr_types.tr_description.str.contains("АТМ")]
result_table = pd.merge(tr_types_sample, random_sample, on='tr_type');
res=len(result_table)/len(random_sample)
res

0.618

## Задание 2 


1. Для столюца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [ ]:
tr_groups = transactions.groupby(['tr_type'])
types_sizes = tr_groups.size()

i = 0
for curr_size in types_sizes:
  curr_frec = curr_size/len(transactions)
  print(types_sizes.index[i], " : " ,curr_frec)
  i+=1

print("\nTop-10 transatcions:")

top10 = types_sizes.sort_values(ascending=False).head(10)
top10

1000  :  8e-06
1010  :  0.231117
1030  :  0.118975
1100  :  0.015476
1110  :  0.137658
1200  :  0.010419
1210  :  0.005244
1310  :  0.000166
1410  :  0.000179
1510  :  2e-06
2010  :  0.151166
2011  :  0.00688
2020  :  0.00179
2021  :  5.7e-05
2100  :  1.9e-05
2110  :  0.001273
2200  :  6e-06
2210  :  0.00068
2320  :  0.001237
2330  :  0.009428
2331  :  0.002697
2340  :  1.4e-05
2341  :  4e-06
2370  :  0.04983
2371  :  0.006264
2440  :  0.000555
2446  :  4e-06
2456  :  6e-05
2460  :  0.000298
4010  :  0.004925
4011  :  0.001846
4020  :  3.6e-05
4021  :  1.1e-05
4031  :  6.3e-05
4041  :  6.8e-05
4051  :  0.002697
4061  :  4e-06
4071  :  0.006264
4090  :  0.000144
4096  :  4e-06
4097  :  5e-05
4100  :  2.8e-05
4110  :  0.001271
4200  :  6e-06
4210  :  0.000451
4500  :  5.7e-05
6000  :  1.7e-05
6010  :  0.000272
6100  :  0.000328
6110  :  0.001666
6200  :  0.000344
6210  :  3.3e-05
7010  :  0.028972
7011  :  0.001252
7014  :  0.000475
7015  :  8e-06
7020  :  0.002036
7021  :  0.000541
7024

tr_type
1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
2370     49830
7010     28972
7030     26078
7071     15781
1100     15476
dtype: int64

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [ ]:
client_with_max_pos_amount = transactions[transactions.amount > 0].groupby("customer_id").sum()
client_with_gender = transactions[transactions.customer_id ==  client_with_max_pos_amount["amount"].idxmax()]
client = transactions[transactions['customer_id'] == 70780820]
print("Пол клиента: ",client['gender'].values[0])
t = transactions.groupby("gender").groups

t1 = transactions[transactions.amount > 0].groupby('gender').groups
t1.get(0)

Пол клиента:  1.0


Int64Index([120396, 120397, 121191, 121222, 121250, 121265, 121267, 121336,
            121339, 121340,
            ...
            998280, 998281, 998283, 998286, 998301, 998304, 999570, 999574,
            999575, 999576],
           dtype='int64', length=43303)

In [ ]:
client_with_max_neg_amount = transactions[transactions.amount < 0].groupby("customer_id").sum()
transactions[transactions.customer_id ==  client_with_max_neg_amount["amount"].idxmin()]

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
613348,70780820,3 08:57:14,6011,2010,-3368873.66,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
613349,70780820,8 14:01:12,6011,2010,-4491831.54,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
613350,70780820,8 14:02:60,6011,2010,-2245915.77,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
613351,70780820,14 12:08:30,6011,2010,-965743.78,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
613352,70780820,17 02:32:56,6011,2010,-1033121.26,NaN,1.0,Финансовые институты — снятие наличности автом...,Выдача наличных в АТМ Сбербанк России
...,...,...,...,...,...,...,...,...,...
997962,70780820,245 11:26:04,6010,7040,14396.32,370359,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997963,70780820,333 02:07:47,6010,7040,19269.96,152770,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997964,70780820,369 11:41:37,6010,7040,10398.59,150547,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...
997965,70780820,374 08:16:14,6010,7040,54396.08,150318,1.0,Финансовые институты — снятие наличности вручную,Перевод на карту (с карты) через POS (в предел...


In [ ]:
res = abs(client_with_max_neg_amount["amount"].values.min() - client_with_max_pos_amount["amount"].values.max())
res

2498067091.5999985

## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

In [ ]:
top10_df = top10.to_frame();
top10_amount = pd.merge(top10_df, transactions, on='tr_type').groupby('tr_type');

for key, group in top10_amount:
  print("Avg for", key, ":", group['amount'].mean())
  print("Median for", key, ":", group['amount'].median())

print("\n\n\n")

client_from_task3_types = transactions[transactions.customer_id == client_with_max_pos_amount["amount"].idxmax()].groupby('tr_type')

for key, group in client_from_task3_types:
  print("Avg for", key, ":", group['amount'].mean())
  print("Median for", key, ":", group['amount'].median())


Avg for 1010 : -19784.748639648507
Median for 1010 : -7411.52
Avg for 1030 : -5320.980221811316
Median for 1030 : -2245.92
Avg for 1100 : -44061.82726156657
Median for 1100 : -10188.259999999998
Avg for 1110 : -32119.330370770458
Median for 1110 : -11207.57
Avg for 2010 : -136077.62932505234
Median for 2010 : -44918.32
Avg for 2370 : -205418.24903167944
Median for 2370 : -44918.32
Avg for 7010 : 276391.7895961331
Median for 7010 : 112295.79
Avg for 7030 : 86104.33290896451
Median for 7030 : 13951.52
Avg for 7070 : 65569.83170018934
Median for 7070 : 11319.42
Avg for 7071 : 66806.82662251407
Median for 7071 : 3593.47




Avg for 2010 : -2941055.8709415505
Median for 2010 : -3368873.66
Avg for 2330 : -2382398.344615385
Median for 2330 : -2245915.77
Avg for 2370 : -2218504.987086618
Median for 2370 : -2245915.77
Avg for 6110 : 17562.93173076922
Median for 6110 : 10028.01
Avg for 7020 : 14651.290741626804
Median for 7020 : 10319.985
Avg for 7021 : 28500.67
Median for 7021 : 28500.67
Avg fo

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
transactions_by_gender_neg_amount = transactions[transactions['amount'] < 0].groupby('gender');
males_amount = 0;
females_amount = 0;


for key, group in transactions_by_gender_neg_amount:
  if (key == 0.0):
    males_amount = group['amount'].mean()
  if (key == 1.0):
    females_amount = group['amount'].mean()

res = abs(males_amount - females_amount)
print("1.Abs. amount diff. of neg. amount: ", res)


transactions_by_gender_pos_amount = transactions[transactions['amount'] > 0].groupby('gender');

for key, group in transactions_by_gender_pos_amount:
  if (key == 0.0):
    males_amount = group['amount'].mean()
  if (key == 1.0):
    females_amount = group['amount'].mean()

res = abs(males_amount - females_amount)
print("2.Abs. amount diff. of pos. amount: ", res)


1.Abs. amount diff. of neg. amount:  32718.05492021877
2.Abs. amount diff. of pos. amount:  63366.57104801515


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте ночные поступления для мужчин и женщин. Ночные - это примерно 00-06 часов. 

In [ ]:
time_df = transactions['tr_datetime'].str.split(" ", expand=True)
time_df.columns = ['tr_day', 'hms']

HMS_df = time_df['hms'].str.split(":", expand=True)
HMS_df.columns = ['tr_hours', 'tr_minutes', 'tr_seconds']
print("1.", HMS_df['tr_hours'],"\n\n\n")

changed_transactions = pd.concat([transactions,HMS_df], axis=1)
grouped_by_gender_transactions = changed_transactions[changed_transactions['tr_hours'].astype('int64') < 7].groupby('gender') 
grouped_by_gender_transactions.size()

1. 0         10
1         07
2         07
3         08
4         14
          ..
999579    23
999580    19
999581    22
999582    23
999583    23
Name: tr_hours, Length: 999584, dtype: object 





gender
0.0    47569
1.0    79420
dtype: int64